In [1]:
from utils.fasta_utils import *
from utils.proteome_process import *
from utils.prefetcher import *
from utils.network_utils import *

import os
from tqdm import tqdm
import pandas as pd
import hdbscan
import faiss
from sklearn.metrics.pairwise import cosine_distances,euclidean_distances
import numpy as np
import networkx as nx



In [2]:
data_folder = "/media/microscopie-lcb/swapnesh/protein/embeddings/phages/1Sept2024_INPHARED_db_latest/"

model_name = "650m"
db_zarr_path = os.path.join(data_folder , f"ESM2_{model_name}_1Sept24_{model_name}.zarr")
#db_accession_path = os.path.join(data_folder , 'db_accessions.npy')
faiss_index_path = os.path.join(data_folder , f"ESM2_{model_name}_1Sept24_{model_name}.zarrfaiss_index.bin")
eps_values_path = os.path.join(data_folder , f"ESM2_{model_name}_1Sept24_{model_name}_eps_values_flat_clusters.npy")
#HieVi_tree =  os.path.join(data_folder , "ESM2_3b_1Sept24_3b.zarr.gexf")
HieVi_INPHARED_ordered_annotation = "HieVi_INPHARED_ordered_annotation.csv"


#htree = nx.read_gexf(HieVi_tree)
db_zarr_store = zarr.open(db_zarr_path,'r')
db_accessions = db_zarr_store['accessions'][:]


#htree = nx.read_gexf(HieVi_tree)
#db_accessions = np.load(db_accession_path)

annotations = pd.read_csv(HieVi_INPHARED_ordered_annotation)
annotations= annotations[annotations["Accession"].isin(db_accessions)]
annotations = annotations.set_index("Accession").loc[db_accessions].reset_index()

eps_values = np.load(eps_values_path)
index = faiss.read_index(faiss_index_path)


/home/swapnesh/anaconda3/envs/esmfold/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2,20,29,33,40,57,62,63,73) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [57]:
#filename = "/media/microscopie-lcb/swapnesh/protein/embeddings/phages/NewGenomes/HieVi_test_case.faa"
filename = "/home/swapnesh/Downloads/TEST_concatenated.fasta"
filename = "/home/swapnesh/Downloads/subset.fasta"

# @title Compute phage representations
expt_name = "TEST"  # @param {type:"string"}
expt_name = expt_name.replace(' ','_')
output_folder = os.path.dirname(filename) + os.sep 
fasta_path = filename
#model_name = "3b" # This colab works for 650m only
mode = "mean"
query_zarr_path = os.path.join(output_folder,f"{expt_name}_{model_name}.zarr")
#!python GenPhageRepresentationsESM2.py {expt_name} {output_folder} {fasta_path} {model_name} {mode}


In [58]:
query_zarr_path = f"{output_folder}{expt_name}_{model_name}.zarr"
print(query_zarr_path)
query_zarr_store = zarr.open(query_zarr_path,'r')
query_vectors = query_zarr_store['vectors_mean'][:]*1.0
query_accessions = query_zarr_store['accessions'][:]


/home/swapnesh/Downloads/TEST_650m.zarr


In [59]:
eps_values,query_vectors.shape

(array([0.00389589, 0.01027566, 0.01244868, 0.01821554, 0.01860557,
        0.02378739, 0.02420528, 0.02531965, 0.02565396, 0.02662903,
        0.02710264, 0.0286349 ]),
 (2, 1280))

In [60]:
radius = 0.0016

# Perform range search
lims, distances, indices = index.range_search(query_vectors, radius)
indices = [indices[lims[i]:lims[i+1]] for i in range(len(query_vectors)) ]
distances = [distances[lims[i]:lims[i+1]] for i in range(len(query_vectors)) ]
indices,distances

([array([  281,   435,   478,   506,   509,   531,   536,   537,   543,
           585,   626,   894,   909,   926,  1030,  1133,  1134,  1135,
          1185,  1218,  1260,  1478,  1666,  1673,  1717,  1734,  1739,
          1752,  1818,  1889,  1890,  1891,  1978,  1988,  2047,  2048,
          2049,  2090,  2107,  5001,  5104,  5105,  5106,  5107,  5108,
          5109,  5110,  5111,  5112,  5113,  5114,  5115,  5116,  5117,
          5118,  5119,  5120,  5121,  5130,  5131,  5132,  5133,  5134,
          5135,  5136,  5138,  5139,  5141,  5142,  5143,  5144,  5145,
          5154,  5155,  5156,  5158,  5159,  5160,  5161,  5162,  5168,
          5208,  5430,  5467,  5784,  5824,  5878,  5895,  5905,  6041,
          6153,  6277,  6334,  6628,  6662,  6663,  6664,  6665,  6687,
          7321,  7443,  7556,  7648,  7917,  8049,  8096,  8210,  8211,
          8215,  8277,  8278,  8279,  8288,  8289,  8362,  8388,  8403,
          8409,  8444,  8449,  8911,  9209,  9369,  9376,  9393,

In [61]:
index.search(query_vectors, 1)


(array([[0.],
        [0.]], dtype=float32),
 array([[ 2090],
        [10221]]))

In [43]:
hievi_cluster_prefix = 'HC_p'
k_neighbours = 16 # 
distance_threshold = 0.023 # 
distance_in_tree = 2

distances, indices = index.search(query_vectors, k_neighbours)
valid_idx = distances[:,0] < distance_threshold
invalid_idx = np.logical_not(valid_idx)

if len(np.where(invalid_idx)[0]):
    print('Cannot classifiy: ' ,len(np.where(invalid_idx)[0]))
    invalid_query_df = pd.DataFrame({"Accession": query_accessions[invalid_idx]})
    invalid_query_df.to_csv(query_zarr_path[:-5] + "_HieVi_Unclassifieds.csv")

D1 = np.squeeze(distances[valid_idx,0])
I1 = np.squeeze(indices[valid_idx,0])

all_indices = np.unique(np.array(I1))
all_nearest_accessions = db_accessions[all_indices]

new_accession = []
for a in all_nearest_accessions:
    new_accession += find_predecessor_and_leaves(htree, a,distance_in_tree)
    

all_nearest_accessions = np.unique(np.ravel(np.array(new_accession)))
all_indices = np.array([np.where(db_accessions==a)[0] for a in all_nearest_accessions])

subset_db_vectors = np.array([index.reconstruct(int(a)) for a in all_indices])
subset_db_vectors.shape


NameError: name 'htree' is not defined

In [48]:
hievi_cluster_prefix = 'HC_'
k_neighbours = 8 # 
distance_threshold = 0.023 # 
distance_in_tree = 2

distances, indices = index.search(query_vectors, k_neighbours)
valid_idx = distances[:,0] < distance_threshold
invalid_idx = np.logical_not(valid_idx)

if len(np.where(invalid_idx)[0]):
    print('Cannot classifiy: ' ,len(np.where(invalid_idx)[0]))
    invalid_query_df = pd.DataFrame({"Accession": query_accessions[invalid_idx]})
    invalid_query_df.to_csv(query_zarr_path[:-5] + "_HieVi_Unclassifieds.csv")

all_indices = np.unique(np.ravel(np.array(indices[valid_idx])))
subset_db_vectors = np.array([db_zarr_store['vectors_mean'][i] for i in all_indices])
distances, indices = index.search(subset_db_vectors, k_neighbours)


D1 = np.squeeze(distances)
I1 = np.squeeze(indices)

all_indices = np.unique(np.array(I1))
all_nearest_accessions = db_accessions[all_indices]

#subset_db_vectors = db_zarr_store['vectors_mean'][all_indices,:]

subset_db_vectors = np.array([db_zarr_store['vectors_mean'][i] for i in all_indices])

print(f"Loaded {subset_db_vectors.shape[0]} Accessions from database." )

Loaded 102 Accessions from database.


In [49]:
# @title Combine query and nearest phages from database data

# Get nearest accessions in tree
nearest_accessions = annotations[annotations["Accession"].isin(all_nearest_accessions)]
nearest_accessions = nearest_accessions.set_index("Accession").loc[all_nearest_accessions].reset_index()

# Combine query and database data
query_df = pd.DataFrame({"Accession": query_accessions[valid_idx],"Query": "yes"})
annotation_df = pd.concat([nearest_accessions, query_df], axis=0)
mprs = np.concatenate((subset_db_vectors, query_vectors[valid_idx]), axis=0)

# Perform clustering
# dist_scaled = euclidean_distances(mprs).astype("double")
# clusterer = hdbscan.HDBSCAN(
#     min_cluster_size=2,
#     n_jobs=32,
#     min_samples=1,
#     allow_single_cluster=False,
#     cluster_selection_method="leaf",
#     metric="precomputed",
#     gen_min_span_tree=True
# )
# clusterer.fit(dist_scaled)

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=2,
    min_samples=1,
    allow_single_cluster=False,
    cluster_selection_method="leaf",
    metric="euclidean",
    gen_min_span_tree=True
)
clusterer.fit(mprs)

annotation_df["HieVi_cluster"] = clusterer.labels_
for i,eps in enumerate(eps_values):
    annotation_df[hievi_cluster_prefix+str(i)] = clusterer.dbscan_clustering(cut_distance=eps,min_cluster_size=2)
#annotation_df

import re
# Function to clean text
def clean_text(text):
    text = re.sub(r'[^\w\s-]', '', text)  # Allow dashes by including "-" in the character class
    text = re.sub(r'%', '', text)  # Remove percent signs
    text = text.strip()  # Remove leading/trailing spaces
    return text    
min_lambda = 1/0.023 # @param {type:"slider", min:-1, max:32, step:1}

node_attributes = ['Accession',"Query", 'Virus_Description', 'Realm', 'Kingdom', 'Phylum',
       'Class', 'Order', 'Family', 'Subfamily', 'Genus','Host_Enveloppe',
       'Host_Isolation', 'Host_species', 'Host_order',
       'Host_phylum', 'Molecule_type','HieVi_cluster', 'HC_0', 'HC_1',
       'HC_2', 'HC_3', 'HC_4', 'HC_5', 'HC_6', 'HC_7', 'HC_8', 'HC_9', 'HC_10',
       'HC_11']

df = annotation_df[node_attributes].copy()
# Apply cleaning to column names and all values
df.columns = [clean_text(col) for col in df.columns]
df = df.applymap(lambda x: clean_text(str(x)))  # Apply to all values


min_lambda =13.00 # @param {type:"slider", min:-1, max:32, step:1}
# Create and save network
G = make_network(clusterer, df,min_lambda=min_lambda)
nx.write_gexf(G, query_zarr_path[:-5] + "_HieVi.gexf")

In [50]:

import networkx as nx
import plotly.graph_objects as go
import pydot
import random
import plotly.colors as pc
from networkx.drawing.nx_pydot import graphviz_layout

def plot_hierarchical_graph(G,layout = "radial"):
    if not isinstance(G, nx.DiGraph):
        raise ValueError("Graph must be a directed graph (nx.DiGraph)")

    # Ensure all node names are strings
    G = nx.relabel_nodes(G, {node: str(node) for node in G.nodes()})

    # Fix attributes: Rename "name" -> "name_old" if it exists
    for node in G.nodes():
        if "name" in G.nodes[node]:
            G.nodes[node]["name_old"] = G.nodes[node].pop("name")

    # Remove problematic attributes (lists, dicts, tuples)
    for node in G.nodes():
        for key, value in list(G.nodes[node].items()):
            if isinstance(value, (list, dict, tuple)):
                del G.nodes[node][key]

    # Convert all attributes to strings
    for node in G.nodes():
        for key in G.nodes[node]:
            G.nodes[node][key] = str(G.nodes[node][key])

    try:
        if layout =="radial":
            pos = graphviz_layout(G, prog="twopi")
        else:
            pos = graphviz_layout(G, prog="dot")
    except Exception as e:
        print("Error: Graphviz failed!", e)
        return

    # Get unique genera and assign random colors
    unique_genera = set(G.nodes[node].get("Genus", "Unclassified") for node in G.nodes())
    color_map = {genus: random.choice(pc.sequential.Electric) for genus in unique_genera} #DEFAULT_PLOTLY_COLORS

    # Edge positions
    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y, line=dict(width=1, color='black'),
        hoverinfo='none', mode='lines'
    )

    # Node positions, colors, tooltips
    node_x, node_y, hover_texts = [], [], []
    node_colors, node_borders = [], []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # Get Genus attribute
        genus = G.nodes[node].get("Genus", "Unclassified")

        # Assign colors: Red border for "nan", orange for "Unclassified", others get unique colors
        if genus == "nan":
            node_colors.append("red")  # Node fill color
            node_borders.append("red")  # Node border color
        elif genus == "Unclassified":
            node_colors.append("gray")  # Unclassified nodes
            node_borders.append("black")
        else:
            node_colors.append(color_map.get(genus, "gray"))  # Assign random colormap
            node_borders.append("black")

        # Tooltip text
        attrs = G.nodes[node]
        tooltip = "<br>".join([f"{key}: {value}" for key, value in attrs.items()])
        hover_texts.append(tooltip)

    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers',
        marker=dict(size=15, color=node_colors,
                    line=dict(width=2, color=node_borders)),  # Border color
        hoverinfo="text", hovertext=hover_texts
    )

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title="HieVi",
                        showlegend=False, hovermode="closest",
                        margin=dict(b=0, t=50, l=0, r=0),
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                    ))

    return fig  # Return the figure for further use

In [51]:
#from utils.plotter import *
# Plot the graph for small graphs only
show = True # @param {type:"boolean"}
if len(df) > 2048:
    show = False
fig = plot_hierarchical_graph(G,"radial")
# Save to HTML
fig.write_html(query_zarr_path[:-5] + "_HieVi.html")

# Show the plot only if show=True
if show:
    fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [53]:
def create_subset_fasta(input_fasta, output_fasta, accession_list):
    """
    Create a new multifasta file containing only sequences matching the given accessions.
    
    Args:
        input_fasta (str): Path to the input multifasta file
        output_fasta (str): Path to the output multifasta file
        accession_list (list): List of accession prefixes to include (e.g., ['phage-01', 'phage-02'])
    
    Returns:
        bool: True if successful, False if there was an error
    """
    try:
        # Convert accession list to set for faster lookup
        accession_set = set(accession_list)
        current_header = None
        current_sequence = []
        sequences_to_keep = {}
        
        # Read the input file and collect matching sequences
        with open(input_fasta, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>'):
                    # Store previous sequence if it matches
                    if current_header and current_sequence:
                        full_sequence = ''.join(current_sequence)
                        sequences_to_keep[current_header] = full_sequence
                    
                    # Start new sequence
                    current_sequence = []
                    current_header = line[1:]  # Remove '>'
                    # Check if this header's accession is in our list
                    accession = current_header.split('_')[0]
                    if accession not in accession_set:
                        current_header = None
                elif current_header:
                    current_sequence.append(line)
            
            # Store the last sequence if it matches
            if current_header and current_sequence:
                full_sequence = ''.join(current_sequence)
                sequences_to_keep[current_header] = full_sequence
        
        # Write the matching sequences to output file
        with open(output_fasta, 'w') as f:
            for header, sequence in sequences_to_keep.items():
                f.write(f">new-{header}\n")
                # Split sequence into lines of 60 characters (common FASTA format)
                for i in range(0, len(sequence), 60):
                    f.write(sequence[i:i+60] + "\n")
        
        print(f"Successfully created {output_fasta} with {len(sequences_to_keep)} sequences")
        return True
        
    except FileNotFoundError:
        print(f"Error: Input file '{input_fasta}' not found")
        return False
    except PermissionError:
        print(f"Error: Cannot write to '{output_fasta}' - check permissions")
        return False
    except Exception as e:
        print(f"Error: {str(e)}")
        return False

# Sample parameters
input_file = "/media/microscopie-lcb/swapnesh/protein/embeddings/phages/1Sept2024_INPHARED_db_latest/1Sep2024_vConTACT2_proteins.faa"
output_file = "/media/microscopie-lcb/swapnesh/protein/embeddings/phages/1Sept2024_INPHARED_db_latest/subset.fasta"
accessions = ["OK499988", "OR756193"]

success = create_subset_fasta(input_file, output_file, accessions)


Successfully created /media/microscopie-lcb/swapnesh/protein/embeddings/phages/1Sept2024_INPHARED_db_latest/subset.fasta with 114 sequences


In [2]:
# required files
'''
Required files
faiss_index for the right model
the network gexf

'''




data_folder = "/media/microscopie-lcb/swapnesh/protein/embeddings/phages/1Sept2024_INPHARED_db_latest/"
faiss_index_path = data_folder +"ESM2_650m_1Sept24_650m.zarrfaiss_index.bin"
query_zarr_path = data_folder +"ESM2_650m_1Sept24_650m.zarr"#f"{expt_name}_{model_name}.zarr"
db_clusters_path = data_folder +"ESM2_3b_1Sept24_3b_flat_clusters_3b.csv"
eps_values_path = data_folder +"ESM2_650m_1Sept24_650m_eps_values_flat_clusters.npy"
db_zarr_path = data_folder +"ESM2_650m_1Sept24_650m.zarr"
HieVi_INPHARED_ordered_annotation = "HieVi_INPHARED_ordered_annotation.csv"
HieVi_tree = data_folder + "ESM2_3b_1Sept24_3b.zarr.gexf"

htree = nx.read_gexf(HieVi_tree)

zarr_store = zarr.open(db_zarr_path,'r')
db_accessions = zarr_store['accessions'][:]

annotations = pd.read_csv(HieVi_INPHARED_ordered_annotation)
annotations= annotations[annotations["Accession"].isin(db_accessions)]
annotations = annotations.set_index("Accession").loc[db_accessions].reset_index()

eps_values = np.load(eps_values_path)


index = faiss.read_index(faiss_index_path)

hievi_cluster = pd.read_csv(db_clusters_path)
hievi_cluster= hievi_cluster[hievi_cluster["Accession"].isin(db_accessions)]
hievi_cluster = hievi_cluster.set_index("Accession").loc[db_accessions].reset_index()


In [3]:
filename = "/media/microscopie-lcb/swapnesh/protein/embeddings/phages/NewGenomes/HieVi_test_case.faa"
# @title Compute phage representations
expt_name = "MyPhages"  # @param {type:"string"}
output_folder = os.path.dirname(filename) + os.sep 
fasta_path = filename
model_name = "650m" # This colab works for 650m only
mode = "mean"
#!python GenPhageRepresentationsESM2.py {expt_name} {output_folder} {fasta_path} {model_name} {mode}


In [4]:
query_zarr_path = f"{output_folder}{expt_name}_{model_name}.zarr"

query_zarr_store = zarr.open(query_zarr_path,'r')
query_vectors = query_zarr_store['vectors_mean'][:]*1.0
query_accessions = query_zarr_store['accessions'][:]



(54, 1280)

array([ 0.00258736, -0.00066257,  0.00058419, ..., -0.01147296,
       -0.00221943,  0.00873887], dtype=float32)

In [ ]:
hievi_cluster_prefix = 'HC_p'
k_neighbours = 64 # 
distance_threshold = 0.023 # 

distances, indices = index.search(query_vectors, 1)
valid_idx = distances[:,0] < distance_threshold
invalid_idx = np.logical_not(valid_idx)

if len(np.where(invalid_idx)[0]):
    print('Cannot classifiy: ' ,len(np.where(invalid_idx)[0]))
    invalid_query_df = pd.DataFrame({"Accession": query_accessions[invalid_idx]})
    invalid_query_df.to_csv(query_zarr_path[:-5] + "_HieVi_Unclassifieds.csv")

D = np.squeeze(distances[valid_idx,0])
I = np.squeeze(indices[valid_idx,0])

D1, I1 = index.search(np.array(zarr_store['vectors_mean'])[I]*1.0, k_neighbours)

all_indices = []
for ds,idxs in zip(I1,D1):
    for d,i in zip(ds,idxs):
        if d < distance_threshold*10:
            all_indices +=[i]


all_indices = np.unique(np.array(all_indices+ list(I)))
all_nearest_accessions = db_accessions[all_indices]

new_accession = []
for a in all_nearest_accessions:
    new_accession += find_predecessor_and_leaves(htree, a,2)
    

all_nearest_accessions = np.unique(np.ravel(np.array(new_accession)))
all_indices = np.array([np.where(db_accessions==a)[0] for a in all_nearest_accessions])

subset_db_vectors = np.squeeze(np.array(zarr_store['vectors_mean'])[all_indices]*1.0)
subset_db_vectors.shape


In [40]:
# @title Combine query and nearest phages from database data

# indices = np.unique(np.ravel(indices))
# print(f"Nearest neighbor search completed. Found {len(indices)} unique neighbors.")
# Get nearest accessions in tree
nearest_accessions = annotations[annotations["Accession"].isin(all_nearest_accessions)]
nearest_accessions = nearest_accessions.set_index("Accession").loc[all_nearest_accessions].reset_index()


# Combine query and database data
query_df = pd.DataFrame({"Accession": query_accessions[valid_idx]})
annotation_df = pd.concat([nearest_accessions, query_df], axis=0)
mprs = np.concatenate((subset_db_vectors, query_vectors[valid_idx]), axis=0)

# Perform clustering
# dist_scaled = euclidean_distances(mprs).astype("double")
# clusterer = hdbscan.HDBSCAN(
#     min_cluster_size=2,
#     n_jobs=32,
#     min_samples=1,
#     allow_single_cluster=False,
#     cluster_selection_method="leaf",
#     metric="precomputed",
#     gen_min_span_tree=True
# )
# clusterer.fit(dist_scaled)

clusterer = hdbscan.HDBSCAN(
    min_cluster_size=2,
    min_samples=1,
    allow_single_cluster=False,
    cluster_selection_method="leaf",
    metric="euclidean",
    gen_min_span_tree=True
)
clusterer.fit(mprs)

annotation_df["HieVi_granular_cluster"] = clusterer.labels_
for i,eps in enumerate(eps_values):
    annotation_df[hievi_cluster_prefix+str(i)] = clusterer.dbscan_clustering(cut_distance=eps,min_cluster_size=2)
#annotation_df

In [41]:
# @title Generate Network


min_lambda =13.00 # @param {type:"slider", min:-1, max:32, step:1}
# Create and save network
G = make_network(clusterer, annotation_df,min_lambda=min_lambda)
nx.write_gexf(G, query_zarr_path[:-5] + "_HieVi.gexf")
